In [ ]:
!pip install kagglehub

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mkechinov/ecommerce-behavior-data-from-multi-category-store")

print("Path to dataset files:", path)

In [ ]:
import os
#path = "C:/Users/User/.cache/kagglehub/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store/versions/8"
os.listdir(path)

In [ ]:

path2 = os.path.join(path, '2019-Oct.csv')

In [ ]:
import pandas as pd

# Dask 데이터프레임 생성
ddf = pd.read_csv(path2)

# 랜덤으로 1% 샘플링
sampled_data = ddf.sample(frac=0.01, random_state=42)

print(sampled_data)

In [ ]:
print(sampled_data.head())
print(sampled_data.shape)
print(sampled_data.info())
print(sampled_data.columns)
visitor = sampled_data['user_id'].nunique()
print("visitors: {}".format(visitor))

In [ ]:
import pandas as pd
#sampled_data = ddf.sample(frac=0.01, random_state=42)
sampled_data=sampled_data.loc[sampled_data.event_type == 'purchase']
sampled_data['event_time'] = pd.to_datetime(sampled_data['event_time']).dt.tz_convert(None)

In [ ]:
sampled_data.reset_index(drop=True, inplace=True)


In [ ]:
sampled_data.dtypes

In [ ]:
usecols=['event_time','event_type','price','user_id','user_session']
product_data = sampled_data.copy()
sampled_data = sampled_data[usecols]

In [ ]:
sampled_data.head()

In [ ]:
print([(column,sampled_data[column].nunique()) for column in sampled_data.columns])

In [ ]:
visitor_by_date = sampled_data[['user_id', 'date']].drop_duplicates().groupby('date').count().sort_values(by='date', ascending=True)

In [ ]:
visitor_by_date

###날짜별 고객수 추이

In [ ]:
import matplotlib.pyplot as plt
x= pd.Series(visitor_by_date.index.values)
y= visitor_by_date["user_id"]
plt.rcParams["figure.figsize"]=(20,8)
plt.plot(x,y)
plt.xticks(rotation = 45)

In [ ]:
product_by_date = sampled_data[['product_id', 'date']].groupby('date').count().sort_values(by='date', ascending=True) # 날짜별 product 갯수
price_by_date = sampled_data[['price', 'date']].groupby('date').mean().sort_values(by='date', ascending=True) # 날짜별 price 평균

### 날짜별 총판매 추이


In [ ]:
total_profit=pd.DataFrame(product_by_date['product_id']*price_by_date['price'])
total_profit.columns=['total_profit']
plt.rcParams["figure.figsize"]=(20,8)
plt.plot(total_profit.index.values, total_profit['total_profit'])

In [ ]:
purchase_data = sampled_data.loc[sampled_data['event_type'] == 'purchase',:]

In [ ]:
purchase_data.reset_index(drop=True).head(5)

In [ ]:
product_price_map = purchase_data.groupby('product_id')['price'].mean().reset_index()


In [ ]:
product_price_map

In [ ]:
top_10_df = purchase_data.groupby('product_id').count().reset_index()[['product_id', 'date']]

In [ ]:
top_10_df.columns = ['product_id', 'count']
top_10_df

In [ ]:
top_10_df = pd.merge(top_10_df, product_price_map, on='product_id', how='outer')
top_10_df['total_profit'] = top_10_df['count'] * top_10_df['price']
top_10_df.sort_values(by='total_profit', ascending=False, inplace=True)
top_10_df.head(10)

In [ ]:
top_10_df.isnull().sum()
top_10_df.reset_index(drop=True, inplace=True)

In [ ]:
purchase_data

In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(20, 10))
axes = axes.flatten()
for idx, product_id in enumerate(top_10_df.loc[:9,"product_id"]):
    product_daily_price = purchase_data.loc[purchase_data['product_id'] == product_id, ['date', 'price']].groupby(['date']).mean()

        # x, y 값 생성
    x = pd.Series(product_daily_price.index.values)
    y = product_daily_price['price']

    axes[idx].plot(x, y, marker='o')
    axes[idx].set_title(f"Product ID: {product_id}")
    axes[idx].set_xlabel("Date")
    axes[idx].set_ylabel("Price")
    axes[idx].tick_params(axis='x', rotation=45)

# 레이아웃 조정 및 출력
plt.tight_layout()
plt.show()

In [ ]:
purchase_data['event_time'] = pd.to_datetime(purchase_data['event_time']).dt.tz_convert(None)
purchase_data['event_time'].min(), purchase_data['event_time'].max()

In [ ]:
purchase_data['event_type']=purchase_data['event_type'].astype('category')
purchase_data['category_code']=purchase_data['category_code'].astype('category')

In [ ]:
usecols=['event_time','event_type','price','user_id','user_session']

In [ ]:
purchase_data = purchase_data[usecols]

In [ ]:
purchase_data.dtypes

In [ ]:
purchase_data=purchase_data[usecols].groupby('user_session').agg(Date_order=('event_time',lambda x: x.max()),
                                                  user_id=('user_id',lambda x: x.unique()),
                                          Quantity=('user_session','count'),
                                         money_spent=('price','sum')).reset_index(drop=True)

In [ ]:
import datetime as dt
import numpy as np
study_date = dt.datetime(2019,11,1)
purchase_data=pd.DataFrame(purchase_data)
purchase_data['last_purchase']=study_date - purchase_data['Date_order']
purchase_data['last_purchase'].astype('timedelta64[D]')
purchase_data['last_purchase']=purchase_data['last_purchase'] / np.timedelta64(1, 'D')
purchase_data.head()

In [ ]:
recency_df.sort_values(by='Recency', ascending=False)

In [ ]:
import seaborn as sns
%matplotlib inline
plt.style.use(['dark_background'])
sns.histplot(data = recency_df, x ='Recency')
plt.gcf().set_size_inches(16.5, 3)

In [ ]:
frequency_df

In [ ]:
frequency_df = purchase_data.copy()
frequency_df.drop_duplicates(subset=['user_id', 'user_session'],keep = "first", inplace=True)
frequency_df = frequency_df.groupby("user_id", as_index = False)["user_session"].count() # 중복되는 값중 제일 첫번째로 나온 값이 출력
frequency_df.columns = ['user_id', 'Frequency']
frequency_df.sort_values(by= 'Frequency',ascending=False).head()


In [ ]:
def extract_categorycode(input_text,level=0):
    '''
     this function splits category code and returns the first part.
    '''
    output_text=input_text.split('.')[level]
    return output_text

def create_clusters(input_data_frame,input_columns,n_cluster):
    '''
     This function creates clusters and cluster labels.
    '''
    from sklearn.cluster import KMeans
    X=input_data_frame[input_columns].values
    k_means=KMeans(n_clusters=n_cluster,random_state=15).fit(X)
    return k_means.labels_

In [ ]:
#creating a summary table for general overview
daily_summary_table=sampled_data.groupby(by=[sampled_data['event_time'].dt.normalize()]).agg(Number_of_daily_visits=('user_session',lambda x: x.nunique()),
                                                                                  Number_of_daily_visitors=('user_id',lambda x: x.nunique())
                                                                                  )
sales_filter=sampled_data['event_type']=='purchase'
sales=sampled_data.loc[sales_filter].groupby(by=[sampled_data['event_time'].dt.normalize()]).agg(number_of_daily_sales=('event_type','count'),
                                                                                      Total_daily_sales=('price','sum')
                                                                                      ).reset_index()
daily_summary_table=pd.merge(left=daily_summary_table,
                          right=sales,
                          left_on=['event_time'],
                          right_on=['event_time'],
                          how='left')
daily_summary_table['conversion_rate']=daily_summary_table['number_of_daily_sales']/daily_summary_table['Number_of_daily_visits']

In [ ]:
sampled_data.head()

In [ ]:
id_count =sampled_data.groupby(by='user_session')['user_id'].count()
id_count = id_count[id_count>1]
id_count

In [ ]:
sampled_data.drop_duplicates(subset=['user_id', 'user_session'],keep = "first", inplace=True)


In [ ]:
user_id=sampled_data.groupby(by='user_session').agg(user_id=('user_id',lambda x: [x.unique()]))


In [ ]:
greater_than_one = user_id[user_id['user_session'].apply(lambda x: len(x[0])) > 1]

# 결과 출력
print(greater_than_one)

In [ ]:
sampled_data=sampled_data.groupby(by='user_session').agg(Date_order=('event_time',lambda x: x.max()),
                                                  user_id=('user_id',lambda x: x),
                                          Quantity=('user_session','count'),
                                         money_spent=('price','sum')).reset_index(drop=True)
sampled_data

In [ ]:
import datetime as dt
import numpy as np
study_date = dt.datetime(2019,11,1)
sampled_data=pd.DataFrame(sampled_data)
sampled_data['last_purchase']=study_date - sampled_data['Date_order']
sampled_data['last_purchase']=sampled_data['last_purchase'] / np.timedelta64(1, 'D')
sampled_data.head()

In [ ]:
print(sampled_data.dtypes)

In [ ]:
RFM= sampled_data.groupby('user_id').agg(Recency=('last_purchase',lambda x: x.min()),
                                 Frequency=('user_id',lambda x: x.count()),
                                 Monetary=('money_spent',lambda x: x.sum()))
RFM.head()

In [ ]:
len(RFM)

In [ ]:
total_df = pd.merge(product_data, RFM, on='user_id', how='inner')
total_df.head()

In [ ]:
total_df['category_code'].str.split('.',n=1).str[1]

In [ ]:
total_df['main_category']=total_df['category_code'].str.split('.').str[0]
total_df['main_category'] = total_df['main_category'].astype('category')
total_df['sub_category']=total_df['category_code'].str.split('.',n=1).str[1]
total_df['sub_category'] = total_df['sub_category'].astype('category')
total_df.drop(columns=['category_code'], inplace=True)
total_df

In [ ]:
category_list = total_df['category_code'].value_counts().index.tolist()
main_category_list = []
for main_category in category_list:
    main_category_list.append(main_category.split('.')[0])
set(main_category_list)

In [ ]:
# 'main_category', 'sub_category', 'brand' 열에 'none' 카테고리 추가
for col in ['main_category', 'sub_category', 'brand']:
    if total_df[col].dtype.name == 'category':
        total_df[col] = total_df[col].cat.add_categories("none")

# NaN 값을 'none'으로 채우기
total_df[['main_category', 'sub_category', 'brand']] = total_df[['main_category', 'sub_category', 'brand']].fillna("none")

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
def extract_categorycode(input_text,level=0):
    '''
     this function splits category code and returns the first part.
    '''
    output_text=input_text.split('.')[level]
    return output_text

def create_clusters(input_data_frame,input_columns,n_cluster):
    '''
     This function creates clusters and cluster labels.
    '''
    from sklearn.cluster import KMeans
    X=input_data_frame[input_columns].values
    k_means=KMeans(n_clusters=n_cluster,random_state=42).fit(X)
    return k_means.labels_

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
# 날짜 관련 변수 생성
total_df['event_date'] = pd.to_datetime(total_df['event_time'])
total_df['hour'] = total_df['event_date'].dt.hour
total_df['day'] = total_df['event_date'].dt.day
total_df['weekday'] = total_df['event_date'].dt.weekday

# 범주형 데이터 인코딩
total_df['event_type'] = LabelEncoder().fit_transform(total_df['event_type'])
total_df['brand'] = LabelEncoder().fit_transform(total_df['brand'])
total_df['main_category'] = LabelEncoder().fit_transform(total_df['main_category'])
total_df['sub_category'] = LabelEncoder().fit_transform(total_df['sub_category'])

# 연속형 변수 스케일링
scaler = MinMaxScaler()
total_df['price'] = scaler.fit_transform(total_df[['price']])
total_df['recency'] = scaler.fit_transform(total_df[['Recency']])
total_df['frequency'] = scaler.fit_transform(total_df[['Frequency']])
total_df['monetary'] = scaler.fit_transform(total_df[['Monetary']])


total_df.head()

In [ ]:
total_df['product_id'] = LabelEncoder().fit_transform(total_df['product_id'])
total_df['category_id'] = LabelEncoder().fit_transform(total_df['category_id'])
total_df.drop(columns=['user_id','user_session'], inplace=True)
total_df